<a href="https://colab.research.google.com/github/didi64/Colab_Test/blob/main/Test_Sentiment_analysis_with_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Exercise 1
Consider only reviews, where
- the positive and negative part consists of at least 10 words,
- the reviewer's nationality is  'United Kingdom',
- the hotel has atleast 300 reviews.

Select n random reviews from each hotel and write  these reviews to a file.

In [ ]:
import pandas as pd
import csv
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
SEED = 123

Mounted at /content/drive


In [ ]:
root = '/content/drive/My Drive/CAS_Gregi/'
fn = root + 'data/Hotel_Reviews.csv'
df = pd.read_csv(fn)
df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [ ]:
def get_selection(df, reviewer_nationality = 'United Kingdom', min_review_len = 10, min_n_review = 300):
    '''returns a df with reviews where
        - the positive and negative part consists of at least min_review_len words,
        - the reviewer's nationality is  reviewer_nationality,
        - the hotel has atleast min_n_review reviews.

       result is a df, sorted by the added col  'Total_Reviews_for_Hotel'
    '''
    # cols that are returned
    cols = ['Hotel_Name', 'Negative_Review', 'Positive_Review', 'Reviewer_Score']

    cond1 = df['Reviewer_Nationality'].str.contains('United Kingdom', case = False)
    cond2 = df['Review_Total_Negative_Word_Counts'] > min_review_len
    cond3 = df['Review_Total_Positive_Word_Counts'] > min_review_len

    pre_selection = df[cond1 & cond2 & cond3]
    pre_selection._is_copy = None # to avoid false SettingWithCopyWarning

    pre_selection['Total_Reviews_for_Hotel'] = pre_selection.groupby('Hotel_Name')['Hotel_Name'].transform('size')
    selection = pre_selection[pre_selection['Total_Reviews_for_Hotel'] >= min_n_review]\
                .sort_values('Total_Reviews_for_Hotel', ascending=False)

    return selection[cols]

def get_random_reviews(selection, n = 10, seed = SEED):
    '''returns n random reviews from each hotel as list of lists'''

    reviews = [list(selection.columns)]

    hotels = set(selection['Hotel_Name'])

    for hotel in  hotels:
        df_hotel = selection[selection['Hotel_Name'] ==  hotel]
        df_reviews = df_hotel.sample(n=n, random_state = seed)
        reviews.extend(df_reviews.values.tolist())

    return reviews

def reviews2csv(reviews, fn):
    with open(fn, mode = 'w', newline = '') as f:
        writer = csv.writer(f, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        for row in reviews:
            writer.writerow(row)

In [ ]:
fn = root + 'data/some_reviews.csv'
cols = ['Hotel_Name', 'Reviewer_Nationality', 'Negative_Review',
          'Review_Total_Negative_Word_Counts', 'Positive_Review',
          'Review_Total_Positive_Word_Counts','Reviewer_Score']

selection = get_selection(df[cols])
reviews = get_random_reviews(selection, n = 10)
reviews2csv(reviews, fn)

In [ ]:
df1 =   pd.read_csv(fn)
df1.head()